# 📓 Manual State-Level County Employment Dataset (BLS LAUS)

This notebook demonstrates how to build a **county-level dataset** for a single state (e.g., California) using BLS Local Area Unemployment Statistics (LAUS) files.

You can change the state FIPS code and data file at the top to process a different state.


In [1]:
# # --- 1. Config ---
# STATE_FIPS = "06"  # California
# DATA_FILE = "la.data.11.California"  # California LAUS data file
# # Example: set STATE_FIPS = "26", DATA_FILE = "la.data.29.Michigan" for Michigan


In [18]:
# --- 1. Config ---
# Define FIPS codes and data file names manually
STATE_CONFIGS = [
    {"fips": "01", "file": "la.data.7.Alabama"},
    {"fips": "02", "file": "la.data.8.Alaska"},
    {"fips": "04", "file": "la.data.9.Arizona"},
    {"fips": "05", "file": "la.data.10.Arkansas"},
    {"fips": "06", "file": "la.data.11.California"},
    {"fips": "08", "file": "la.data.12.Colorado"},
    {"fips": "09", "file": "la.data.13.Connecticut"},
    {"fips": "10", "file": "la.data.14.Delaware"},
    {"fips": "11", "file": "la.data.15.DC"},
    {"fips": "12", "file": "la.data.16.Florida"},
    {"fips": "13", "file": "la.data.17.Georgia"},
    {"fips": "15", "file": "la.data.18.Hawaii"},
    {"fips": "16", "file": "la.data.19.Idaho"},
    {"fips": "17", "file": "la.data.20.Illinois"},
    {"fips": "18", "file": "la.data.21.Indiana"},
    {"fips": "19", "file": "la.data.22.Iowa"},
    {"fips": "20", "file": "la.data.23.Kansas"},
    {"fips": "21", "file": "la.data.24.Kentucky"},
    {"fips": "22", "file": "la.data.25.Louisiana"},
    {"fips": "23", "file": "la.data.26.Maine"},
    {"fips": "24", "file": "la.data.27.Maryland"},
    {"fips": "25", "file": "la.data.28.Massachusetts"},
    {"fips": "26", "file": "la.data.29.Michigan"},
    {"fips": "27", "file": "la.data.30.Minnesota"},
    {"fips": "28", "file": "la.data.31.Mississippi"},
    {"fips": "29", "file": "la.data.32.Missouri"},
    {"fips": "30", "file": "la.data.33.Montana"},
    {"fips": "31", "file": "la.data.34.Nebraska"},
    {"fips": "32", "file": "la.data.35.Nevada"},
    {"fips": "33", "file": "la.data.36.NewHampshire"},
    {"fips": "34", "file": "la.data.37.NewJersey"},
    {"fips": "35", "file": "la.data.38.NewMexico"},
    {"fips": "36", "file": "la.data.39.NewYork"},
    {"fips": "37", "file": "la.data.40.NorthCarolina"},
    {"fips": "38", "file": "la.data.41.NorthDakota"},
    {"fips": "39", "file": "la.data.42.Ohio"},
    {"fips": "40", "file": "la.data.43.Oklahoma"},
    {"fips": "41", "file": "la.data.44.Oregon"},
    {"fips": "42", "file": "la.data.45.Pennsylvania"},
    {"fips": "44", "file": "la.data.47.RhodeIsland"},
    {"fips": "45", "file": "la.data.48.SouthCarolina"},
    {"fips": "46", "file": "la.data.49.SouthDakota"},
    {"fips": "47", "file": "la.data.50.Tennessee"},
    {"fips": "48", "file": "la.data.51.Texas"},
    {"fips": "49", "file": "la.data.52.Utah"},
    {"fips": "50", "file": "la.data.53.Vermont"},
    {"fips": "51", "file": "la.data.54.Virginia"},
    {"fips": "53", "file": "la.data.56.Washington"},
    {"fips": "55", "file": "la.data.58.Wisconsin"},
    {"fips": "56", "file": "la.data.59.Wyoming"},
]

In [3]:
# # --- 2. Setup ---
# import pandas as pd

# # Load lookup files
# la_area = pd.read_csv("la.area", sep="\t", dtype=str)
# la_area_type = pd.read_csv("la.area_type", sep="\t", dtype=str)

# la_series = pd.read_csv(
#     "la.series", 
#     sep="\t", header=None,
#     names=[
#         "series_id","area_type_code","area_code",
#         "measure_code","seasonal","srd_code",
#         "series_title","footnote_codes",
#         "begin_year","begin_period",
#         "end_year","end_period"
#     ],
#     dtype=str
# )

# print("Lookup files loaded.")

Lookup files loaded.


In [ ]:
import pandas as pd

la_area = pd.read_csv("la.area", sep="\t", dtype=str)
la_area_type = pd.read_csv("la.area_type", sep="\t", dtype=str)

la_series = pd.read_csv(
    "la.series", 
    sep="\t", header=None,
    names=[
        "series_id","area_type_code","area_code",
        "measure_code","seasonal","srd_code",
        "series_title","footnote_codes",
        "begin_year","begin_period",
        "end_year","end_period"
    ],
    dtype=str
)

la_series.head(5)

In [22]:
# # --- 3. Identify Counties for Selected State ---
# state_counties = la_area[
#     (la_area["area_type_code"] == "F") & 
#     (la_area["area_code"].str.startswith("CN" + STATE_FIPS))
# ].copy()

# print(f"Counties found for FIPS {STATE_FIPS}: {len(state_counties)}")
# state_counties.head()

In [24]:
all_datasets = []

for config in STATE_CONFIGS:
    fips = config["fips"]
    data_file = config["file"]
    
    # Identify counties
    state_counties = la_area[
        (la_area["area_type_code"] == "F") &
        (la_area["area_code"].str.startswith("CN" + fips))
    ].copy()
    
    if state_counties.empty:
        print(f"⚠️ No counties found for {data_file}")
        continue
    
    # Load data
    la_data = pd.read_csv(
        data_file,
        sep="\t", header=None,
        names=["series_id","year","period","value","footnote_codes"],
        dtype=str
    )
    
    # Filter series
    state_series = la_series[la_series["area_code"].isin(state_counties["area_code"])].copy()
    
    # Merge
    merged = (
        la_data
        .merge(state_series, on="series_id")
        .merge(state_counties, on="area_code")
    )
    
    if merged.empty:
        print(f"⚠️ Empty merge for {data_file}")
        continue
    
    merged["year"] = merged["year"].astype(int)
    merged["month"] = merged["period"].str[1:].astype(int)
    merged["value"] = pd.to_numeric(merged["value"], errors="coerce")
    
    # Pivot
    pivoted = merged.pivot_table(
        index=["area_text","year","month"],
        columns="measure_code",
        values="value"
    ).reset_index()
    
    pivoted.rename(columns={
        "03": "unemployment_rate",
        "04": "labor_force",
        "05": "employment",
        "06": "unemployment"
    }, inplace=True)
    
    state_final = pivoted.rename(columns={"area_text": "county"})
    all_datasets.append(state_final)
    
    # Save each state separately
    state_final.to_csv(f"state_{fips}_county_employment.csv", index=False)
    print(f"✅ Saved {data_file} with {len(state_final)} rows")

✅ Saved la.data.7.Alabama with 30552 rows
✅ Saved la.data.8.Alaska with 12718 rows
✅ Saved la.data.9.Arizona with 6840 rows
✅ Saved la.data.10.Arkansas with 34200 rows
✅ Saved la.data.11.California with 26449 rows
✅ Saved la.data.12.Colorado with 29184 rows
✅ Saved la.data.13.Connecticut with 4104 rows
✅ Saved la.data.14.Delaware with 1368 rows
✅ Saved la.data.15.DC with 457 rows
✅ Saved la.data.16.Florida with 30553 rows
✅ Saved la.data.17.Georgia with 72504 rows
✅ Saved la.data.18.Hawaii with 1824 rows
✅ Saved la.data.19.Idaho with 20064 rows
✅ Saved la.data.20.Illinois with 46512 rows
✅ Saved la.data.21.Indiana with 41952 rows
✅ Saved la.data.22.Iowa with 45144 rows
✅ Saved la.data.23.Kansas with 47880 rows
✅ Saved la.data.24.Kentucky with 54720 rows
✅ Saved la.data.25.Louisiana with 29100 rows
✅ Saved la.data.26.Maine with 7296 rows
✅ Saved la.data.27.Maryland with 10944 rows
✅ Saved la.data.28.Massachusetts with 6384 rows
✅ Saved la.data.29.Michigan with 37848 rows
✅ Saved la.data

In [7]:
# # --- 4. Load State Data File ---
# la_data = pd.read_csv(
#     DATA_FILE, 
#     sep="\t", header=None,
#     names=["series_id","year","period","value","footnote_codes"],
#     dtype=str
# )

# print(f"Loaded {DATA_FILE}, rows: {len(la_data)}")
# la_data.head()

Loaded la.data.11.California, rows: 691539


,series_id,year,period,value,footnote_codes
0,series_id,year,period,value,footnote_codes
1,LASBS060000000000003,1976,M01,9.4,R
2,LASBS060000000000003,1976,M02,9.3,R
3,LASBS060000000000003,1976,M03,9.3,R
4,LASBS060000000000003,1976,M04,9.2,R


In [9]:
# # --- 5. Filter Series for State Counties ---
# state_series = la_series[la_series["area_code"].isin(state_counties["area_code"])].copy()
# print(f"Series found for state: {len(state_series)}")
# state_series.head()

Series found for state: 232


,series_id,area_type_code,area_code,measure_code,seasonal,srd_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
1951,LAUCN060010000000003,F,CN0600100000000,03,U,06,"Unemployment Rate: Alameda County, CA (U)",NaN,1990,M01,2025,M02
1952,LAUCN060010000000004,F,CN0600100000000,04,U,06,"Unemployment: Alameda County, CA (U)",NaN,1990,M01,2025,M02
1953,LAUCN060010000000005,F,CN0600100000000,05,U,06,"Employment: Alameda County, CA (U)",NaN,1990,M01,2025,M02
1954,LAUCN060010000000006,F,CN0600100000000,06,U,06,"Labor Force: Alameda County, CA (U)",NaN,1990,M01,2025,M02
1955,LAUCN060030000000003,F,CN0600300000000,03,U,06,"Unemployment Rate: Alpine County, CA (U)",NaN,1990,M01,2025,M02


In [11]:
# --- 6. Merge Data + Series + Counties ---
merged = (
    la_data
    .merge(state_series, on="series_id")
    .merge(state_counties, on="area_code")
)

merged["year"] = merged["year"].astype(int)
merged["month"] = merged["period"].str[1:].astype(int)
merged["value"] = pd.to_numeric(merged["value"], errors="coerce")

print("Merged dataset:", len(merged), "rows")
merged.head()

Merged dataset: 105796 rows


,series_id,year,period,value,footnote_codes_x,area_type_code_x,area_code,measure_code,seasonal,srd_code,...,begin_year,begin_period,end_year,end_period,area_type_code_y,area_text,display_level,selectable,sort_sequence,month
0,LAUCN060010000000003,1990,M01,3.9,NaN,F,CN0600100000000,03,U,06,...,1990,M01,2025,M02,F,"Alameda County, CA",0,T,431,1
1,LAUCN060010000000003,1990,M02,3.8,NaN,F,CN0600100000000,03,U,06,...,1990,M01,2025,M02,F,"Alameda County, CA",0,T,431,2
2,LAUCN060010000000003,1990,M03,3.6,NaN,F,CN0600100000000,03,U,06,...,1990,M01,2025,M02,F,"Alameda County, CA",0,T,431,3
3,LAUCN060010000000003,1990,M04,3.8,NaN,F,CN0600100000000,03,U,06,...,1990,M01,2025,M02,F,"Alameda County, CA",0,T,431,4
4,LAUCN060010000000003,1990,M05,3.8,NaN,F,CN0600100000000,03,U,06,...,1990,M01,2025,M02,F,"Alameda County, CA",0,T,431,5


In [13]:
# --- 7. Pivot to Wide Format ---
pivoted = merged.pivot_table(
    index=["area_text","year","month"],
    columns="measure_code",
    values="value"
).reset_index()

pivoted.rename(columns={
    "03": "unemployment_rate",
    "04": "labor_force",
    "05": "employment",
    "06": "unemployment"
}, inplace=True)

state_final = pivoted.rename(columns={"area_text": "county"})
state_final.head()

measure_code,county,year,month,unemployment_rate,labor_force,employment,unemployment
0,"Alameda County, CA",1990,1,3.9,26445.0,657058.0,683503.0
1,"Alameda County, CA",1990,2,3.8,25360.0,650679.0,676039.0
2,"Alameda County, CA",1990,3,3.6,24236.0,649735.0,673971.0
3,"Alameda County, CA",1990,4,3.8,25591.0,645659.0,671250.0
4,"Alameda County, CA",1990,5,3.8,25251.0,646698.0,671949.0


In [15]:
# --- 8. Save Output ---
out_csv = f"state_{STATE_FIPS}_county_employment.csv"
out_xlsx = f"state_{STATE_FIPS}_county_employment.xlsx"

state_final.to_csv(out_csv, index=False)
state_final.to_excel(out_xlsx, index=False)

print(f"✅ Saved {out_csv} and {out_xlsx}")

✅ Saved state_06_county_employment.csv and state_06_county_employment.xlsx
